### PBL2-4
- 웹 서버 로그 데이터를 분석하여 악성 요청을 탐지하는 머신러닝 분류 모델 구축

- 전처리:
    - timestamp --> hour 추출
    - status_code --> is_error, label 생성

- 모델: 로지스틱 회귀 모델

- 평가: Accuracy, Precision, Recall, F1-Score 출력

- 가이드
    - Pandas로 속성 생성
    - Sklearn의 LogisticRegression, classification_report 사용
    - status_code가 400 이상인 경우, Error로 간주

### 필요 기능 구분
- 로지스틱 회귀
    - 데이터 로드
    - 데이터 전처리

- 성능 평가 및 지표 출력
    - 정확도
    - 정밀도
    - 재현율
    - F1-Score
    - classification report


In [180]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import pandas as pd
import ipaddress

In [181]:
# 데이터셋 로드 및 분리
log_data = pd.read_csv('web_server_logs_2.csv', delimiter=',')


In [182]:
log_data.isnull().sum()

ip             0
timestamp      0
method         0
status_code    0
size           0
label          0
dtype: int64

In [183]:
log_data['timestamp'].head()

0    2024-11-30 09:26:01
1    2024-11-30 12:00:34
2    2024-12-01 00:22:12
3    2024-11-30 19:33:26
4    2024-11-30 05:37:26
Name: timestamp, dtype: object

In [184]:
#timestamp --> hour 추출
log_timestamp = log_data['timestamp']
time_hour = pd.to_datetime(log_timestamp)  

In [185]:
print(time_hour)

0      2024-11-30 09:26:01
1      2024-11-30 12:00:34
2      2024-12-01 00:22:12
3      2024-11-30 19:33:26
4      2024-11-30 05:37:26
               ...        
1495   2024-11-30 05:06:59
1496   2024-11-30 17:12:08
1497   2024-11-30 21:50:56
1498   2024-11-30 08:23:04
1499   2024-11-30 21:23:58
Name: timestamp, Length: 1500, dtype: datetime64[ns]


In [186]:
#시간 부분만 추출
log_data['timestamp'] = time_hour.dt.hour
#print(log_data['timestamp'].head())

In [187]:
log_data['status_code'].head()

0    301
1    200
2    200
3    200
4    200
Name: status_code, dtype: int64

In [188]:
#status_code 변환
log_data['status_code'] = log_data['status_code'].apply(lambda x: 'Error' if x>= 400 else 'label')


In [189]:
#log_data['status_code'].head()

In [190]:
#데이터 전처리
log_data_encoded = pd.get_dummies(log_data, columns=['method', 'status_code'])

In [191]:
log_data_encoded.head()

,ip,timestamp,size,label,method_DELETE,method_GET,method_OPTIONS,method_POST,method_PUT,status_code_Error,status_code_label
0,192.168.1.138,9,541,0,False,False,True,False,False,False,True
1,192.168.1.130,12,5239,0,False,False,True,False,False,False,True
2,192.168.1.75,0,5778,0,False,False,False,True,False,False,True
3,192.168.1.176,19,3911,0,True,False,False,False,False,False,True
4,10.0.0.113,5,7765,0,False,True,False,False,False,False,True


In [197]:
#상위 10개 IP
top_ips = log_data['ip'].value_counts().head(20).index
#해당 IP가 포함된 행 필터링
top_ip_data = log_data[log_data['ip'].isin(top_ips)]

ip_label_counts = top_ip_data[top_ip_data['label'] == 1].groupby('ip').size()
print(ip_label_counts.sort_values(ascending=False))



ip
192.168.1.145    6
10.0.0.148       5
10.0.0.112       4
10.0.0.33        4
10.0.0.156       4
10.0.0.81        4
192.168.1.184    3
10.0.0.170       2
192.168.1.36     2
10.0.0.53        1
10.0.0.137       1
192.168.1.15     1
192.168.1.195    1
192.168.1.28     1
192.168.1.30     1
dtype: int64


In [193]:
#log_data_seleted['ip'] = log_data['ip'].apply(lambda x: int(ipaddress.IPv4Address(x)))


In [194]:
#IP와 악성 요청간의 관계가 적다고 판단해 IP열 제거
log_data_seleted = log_data_encoded.drop('ip', axis=1)

In [195]:
scaler = StandardScaler()
log_data_seleted = scaler.fit_transform(log_data_seleted)

In [196]:
#로지스틱 회귀를 위한 데이터 분리
X = log_data_seleted.drop('label', axis=1)
y = log_data_seleted['label']

AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [ ]:
#X.head()
#y.head()

In [ ]:
#학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
#모델 학습
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [ ]:
#테스트 데이터 예측
y_pred = model.predict(X_test)

In [ ]:
#평가 지표 계산
accuracy = accuracy_score(y_test, y_pred) # 정확도 계산
precision = precision_score(y_test, y_pred) # 정밀도 계산
recall = recall_score(y_test, y_pred) # 재현율 계산
f1 = f1_score(y_test, y_pred) # F1-Score 계산

In [ ]:
#결과 출력
print("정확도(Accuracy):", accuracy) # 정확도 출력
print("정밀도(Precision):", precision) # 정밀도 출력
print("재현율(Recall):", recall) # 재현율 출력
print("F1-Score:", f1) # F1-Score 출력

정확도(Accuracy): 0.8933333333333333
정밀도(Precision): 0.8181818181818182
재현율(Recall): 0.8526315789473684
F1-Score: 0.8350515463917526


In [ ]:
#혼동 행렬 출력
print("\n혼동 행렬 (Confusion Matrix):\n", confusion_matrix(y_test, y_pred))



혼동 행렬 (Confusion Matrix):
 [[187  18]
 [ 14  81]]


In [ ]:
#분류 보고서 출력: classification_report()
print("\n분류 보고서 (Classification Report):\n",
classification_report(y_test, y_pred))


분류 보고서 (Classification Report):
               precision    recall  f1-score   support

           0       0.93      0.91      0.92       205
           1       0.82      0.85      0.84        95

    accuracy                           0.89       300
   macro avg       0.87      0.88      0.88       300
weighted avg       0.89      0.89      0.89       300

